In [31]:
from optimade.filter import Parser
from lark import Lark, Transformer

p = Parser(version=(0, 9, 5))
# (x<3  and  y!=2)  or  not  z<0
# $or:  [    {x:  {$lt:  3},  y:  {$ne:  2}},  {z:  {$not:  {$lt:  0}}}    ]    }
# filter=formula_prototype=“AB2C3”  AND  nelements=4  AND  elements=“Si,O”
# {formula_prototype:  "AB2C3”,  nelements:  4,  elements:  {$all:  [“Si”,  “O”]}}
t1 = p.parse("filter= x<3 and y!=2 or not z<0")
print(p)

start
  filter=
  expression
    expression
      term
        term
          atom
            comparison
              x
              <
              3
        and
        atom
          comparison
            y
            !=
            2
    or
    term
      atom
        not
        comparison
          z
          <
          0



In [32]:
switcher = {
    "=": "$eq",
    ">":"$gt",
    ">=":"$gte",
    "<":"$lt",
    "<=":"$lte",
    "!=":"$ne",
    "AND":"$and",
    "and":"$and",
    "not":"$not",
    "NOT":"$not",
    "or":"$or",
    "OR":"$or",
}
def operatorToMongoDBSyntax(operator):
    return switcher[operator]

def listToString(l):
    return ''.join(l)

In [33]:
class EmptyTransformer(Transformer):
    def comparison(self, args):
        return args

In [57]:
class MyTransformer(Transformer):
    def comparison(self, args):
        A = args[0]
        operator = args[1]
        try:
            operator = operatorToMongoDBSyntax(args[1])
        except:
            print("Invalid operator detected: " + operator)
        B = args[2]
        return "{{{}: {{{}: {}}}}}".format(A, operator, B)
    def atom(self, args):
        print(args)
        return args[0]
    def term(self, args):
        if(len(args) == 1):
            # if the term is just an atom
            return args[0]
        elif(len(args) == 3):
            conjunction = operatorToMongoDBSyntax(args[1])
            A = args[0]
            B = args[2]
            #print("{{ {}: [{},{}]}}".format(conjunction, A, B))
            return "{{ {}: [{},{}]}}".format(conjunction, A, B)
        else:
            print(args)
            return "here"
    def expression(self,args):
        return args[0]
    def start(self, args):
        return args[1]

In [58]:
transformer = MyTransformer().transform(p.tree)

['{x: {$lt: 3}}']
['{y: {$ne: 2}}']
[Token(NOT, 'not'), '{z: {$lt: 0}}']


In [59]:
print(transformer)
#{formula_prototype:  "AB2C3”,  nelements:  4,  elements:  {$all:  [“Si”,  “O”]}}

{ $and: [{x: {$lt: 3}},{y: {$ne: 2}}]}


In [189]:
transformer

'{ $and: [{a: {$lt: 0}},{b: {$gte: 3}}]}'

In [3]:
!pip install pql
import pql

  Using cached https://files.pythonhosted.org/packages/f4/01/1d79fb709aae58a3716f64691dae31d42984ac5e0e420fbe21755392dd76/pql-0.4.3.tar.gz
  Using cached https://files.pythonhosted.org/packages/22/69/6f8358c6bbca32f1c94167ccab217bc6a70d17bdb10d4a71f4de96431e6e/pymongo-3.7.1-cp36-cp36m-macosx_10_6_intel.whl
  Running setup.py bdist_wheel for pql ... done
  Stored in directory: /Users/michaelwu/Library/Caches/pip/wheels/4b/15/ef/9a7ebb402f8f74d48a77d8fa82bc34ab55aa1410af1874ea38
Successfully built pql


In [14]:
pql.find("formula_prototyp == 'AB2C3'")

{'formula_prototyp': 'AB2C3'}